## Agile Hardware Design
***
# Combinational Logic

<img src="./images/chisel_1024.png" alt="agile hardware design logo" style="float:right; width: 200px;" />

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* A Bit of parameterization
* Scala/Chisel conditionals
* _Result:_ comfortably construct combinational circuits

## Test Chisel Library ##

In [22]:
// Before we start. We test for UCB stcture and make sure they worked.
// Below, we test for UCB stcture and make sure they worked.
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/003_CombLogic/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/003_CombLogic/001_Code/source/load-ivy.sc"

In [23]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [24]:
// Chisel Code: Declare a new module definition
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Passthrough))

Elaborating design...
Done elaborating.
module Passthrough(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [3:0] io_out
);
  assign io_out = io_in; // @[cmd23.sc 6:10]
endmodule



defined class Passthrough

## Loading The Chisel Library Into a Notebook

In [25]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Chisel Multiplexors (Mux)

* Can explicitly instantiate a _mux_
```scala
Mux(select, in1, in0)
```
* _Note:_ input 1 (true case) is first, analogous to ternary (`?`) from Verilog/C
* More flavors of muxes (e.g. `MuxCase`, `Mux1H`) available in [Chisel Library](https://javadoc.io/doc/edu.berkeley.cs/chisel3_2.13/latest/chisel3/util/Mux1H$.html)

<p>
<img src="images/mux.svg" alt="mux schematic" style="width:60%;margin:auto"/>

In [26]:
class MyMux extends Module {
    val io = IO(new Bundle {
        val s   = Input(Bool())
        val in0 = Input(Bool())
        val in1 = Input(Bool())
        val out = Output(Bool())
    })
    io.out := Mux(io.s, io.in1, io.in0)
}
//printVerilog(new MyMux)
println(getVerilog(new MyMux))

Elaborating design...
Done elaborating.
module MyMux(
  input   clock,
  input   reset,
  input   io_s,
  input   io_in0,
  input   io_in1,
  output  io_out
);
  assign io_out = io_s ? io_in1 : io_in0; // @[cmd25.sc 8:18]
endmodule



defined class MyMux

## A Bit More on Scala's `class`

* Arguments are constructor parameters
* Make a class instance with `new`, and internals are evaluated on instantiation
* Default scope for internals is public
* Arguments need `val` to be made public
* Will cover methods and overloading later

In [27]:
class MyClass(argS: String, argI: Int) {
    val name = argS
    println("Created " + argS)
}
val mc = new MyClass("mc", 4)
// mc.name = "foo"
println(mc.name)
// println(mc.argI)

Created mc
mc


defined class MyClass
mc: MyClass = ammonite.$sess.cmd26$Helper$MyClass@18f50b52

## Parameterizing the Mux Width

* Can use class arguments to parameterize our module
* Recommend using Scala types for parameters, and then casting (if necessary) inside module

<p>
<img src="images/muxw.svg" alt="mux schematic" style="width:60%;margin:auto"/>

In [28]:
class MyPMux(w: Int) extends Module {
    val io = IO(new Bundle {
        val s   = Input(Bool())
        val in0 = Input(UInt(w.W))
        val in1 = Input(UInt(w.W))
        val out = Output(UInt(w.W))
    })
    io.out := Mux(io.s, io.in1, io.in0)
}
//printVerilog(new MyPMux(8))
println(getVerilog(new MyPMux(8)))

Elaborating design...
Done elaborating.
module MyPMux(
  input        clock,
  input        reset,
  input        io_s,
  input  [7:0] io_in0,
  input  [7:0] io_in1,
  output [7:0] io_out
);
  assign io_out = io_s ? io_in1 : io_in0; // @[cmd27.sc 8:18]
endmodule



defined class MyPMux

## Scala `if/else`

* If/else akin to other languages
* _Note:_ due to functional nature of language, if/else returns last value of evaluated clause
* Can omit parenthesis if only one statement for clause
  * If short, style recommends keeping entire if one line

In [29]:
val condition = true
if (condition) {
    println("true case")
} else {
    println("false case")
}
val x = if (condition) 3 else 4
println(x)

true case
3


condition: Boolean = true
x: Int = 3

## Contrasting Conditional Execution/Selection

### In Circuit (Chisel Mux)

* Selects based on actual circuit inputs
* Hardware contains both "ways"

```scala
val absX = Mux(x < 0.S, -x, x)
```

<img src="images/absMux.svg" alt="mux schematic" style="width:55%;margin:auto"/>

### During Generation (Scala if/else)

* Executed path generates hardware
* Depends on generator parameters, not circuit inputs

```scala
val invX = if (invert) -x else x
```

<img src="images/invCond.svg" alt="conditional schematic" style="width:55%;margin:auto"/>

## Chisel Tool Flow Frontend (from lecture 2)

<img src="./images/frontend.svg" alt="Chisel frontend" style="width:80%;margin:auto"/>

* The generated Circuit (`.fir` file) is a specific design instance, and it can be passed off to a _backend_ for simulation or implementation

## Chisel Execution

### Elaborated Hardware design is a useful byproduct of your Chisel program
* Your Chisel design is a Scala program
* As the program executes, under-the-hood it builds up your design using the Chisel Library
* As the program ends, it _elaborates_ (outputs) the design as a firrtl file (concrete instance)

### Core operations in Chisel are simple, Scala combines them productively
* Chisel _components are simple_ things like logic operators, wires, registers, and modules
* To make a design, these components need to be _instantiated_ and _connected_
* Is _productive_ to use Scala to programmatically instantiate and connect components (meta programming)
* Designing with Chisel is programming _spatially_ (creating & connecting components) instead of _temporally_ (conventional software which is about order of operations)

## Scala Values Are References to Chisel Objects

* Our generators are simply instantiating Chisel objects and connecting them together
  * Scala program allows us to control which objects & connections
* The connect operator (`:=`) assigns output of right hand side to input of left hand side
* Can use Scala references to name intermediate results

In [30]:
class MyXOR extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    val myGate = io.a ^ io.b
    io.c := myGate
}
//printVerilog(new MyXOR)
println(getVerilog(new MyXOR))

Elaborating design...
Done elaborating.
module MyXOR(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_c
);
  assign io_c = io_a ^ io_b; // @[cmd29.sc 7:23]
endmodule



defined class MyXOR

<img src="images/xorRef.svg" alt="XOR with Scala references" style="width:55%;margin:auto"/>

## Chisel `Wire`

* Sometimes need to connect things, but don't know both ends simultaneously
* Commonly used with `when` construct (next slide)

In [31]:
class MyXOR2 extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Output(Bool())
    })
    val myWire = Wire(Bool())
    myWire := io.a ^ io.b
    io.c := myWire
}
//printVerilog(new MyXOR2)
println(getVerilog(new MyXOR2))

Elaborating design...
Done elaborating.
module MyXOR2(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_c
);
  assign io_c = io_a ^ io_b; // @[cmd30.sc 8:20]
endmodule



defined class MyXOR2

<img src="images/xorWire.svg" alt="XOR with Chisel Wire" style="width:55%;margin:auto"/>

## Chisel `when`

* When condition is true, performs Chisel operations contained in block
* Generates selection in hardware
  * Under the hood, Chisel will implement with muxes
* Can use `.otherwise` like else
* Can use `.elsewhen` like if else

In [32]:
class LastC extends Module {
    val io = IO(new Bundle {
        val x   = Input(Bool())
        val y   = Output(UInt())
    })
    val w = Wire(UInt())
    w := 1.U
    when (io.x) {
        w := 7.U
    }
    io.y := w
}
//printVerilog(new LastC)
println(getVerilog(new LastC))

Elaborating design...
Done elaborating.
module LastC(
  input        clock,
  input        reset,
  input        io_x,
  output [2:0] io_y
);
  assign io_y = io_x ? 3'h7 : 3'h1; // @[cmd31.sc 8:17 cmd31.sc 9:11 cmd31.sc 7:7]
endmodule



defined class LastC

## Last Connect Example

* Absolute Value using `when`

<img src="images/absMux.svg" alt="mux schematic" style="width:55%;margin:auto"/>

In [33]:
class WhenAbs(w: Int) extends Module {
    val io = IO(new Bundle {
        val x    = Input(SInt(w.W))
        val absX = Output(SInt(w.W))
    })
    io.absX := io.x
    when (io.x < 0.S) {
        io.absX := -io.x
    }
}
//printVerilog(new WhenAbs(4))
println(getVerilog(new WhenAbs(4)))

Elaborating design...
Done elaborating.
module WhenAbs(
  input        clock,
  input        reset,
  input  [3:0] io_x,
  output [3:0] io_absX
);
  wire [3:0] _T_3 = 4'sh0 - $signed(io_x); // @[cmd32.sc 8:20]
  assign io_absX = $signed(io_x) < 4'sh0 ? $signed(_T_3) : $signed(io_x); // @[cmd32.sc 7:23 cmd32.sc 8:17 cmd32.sc 6:13]
endmodule



defined class WhenAbs

## Bitwidth Truncation

* Width inference will set widths based on rules for operator
* Operators like add can truncate (`+`,`+%`) or grow (`+&`) to not loose data
* If widths set, will truncate or grow
  * UInts zero extend
  * SInts sign extend
* Consult [Chisel Cheat Sheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)

In [34]:
class MyAdder(w: Int) extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(w.W))
        val b = Input(UInt(w.W))
        val c = Output(UInt())
    })
    io.c := io.a + io.b
//     io.c := io.a +% io.b
//     io.c := io.a +& io.b
}
//printVerilog(new MyAdder(8))
println(getVerilog(new MyAdder(8)))

Elaborating design...
Done elaborating.
module MyAdder(
  input        clock,
  input        reset,
  input  [7:0] io_a,
  input  [7:0] io_b,
  output [7:0] io_c
);
  assign io_c = io_a + io_b; // @[cmd33.sc 7:18]
endmodule



defined class MyAdder

## Example: Sign & Magnitude -> 2's Complement

In [35]:
class SignMagConv(w: Int) extends Module {
    val io = IO(new Bundle {
        val sign = Input(Bool())
        val mag  = Input(UInt(w.W))
        val twos = Output(UInt((w+1).W))
    })
    when (io.sign) {
        io.twos := ~io.mag +& 1.U
    } .otherwise {
        io.twos := io.mag
    }
}

defined class SignMagConv

In [36]:
//printVerilog(new SignMagConv(7))
println(getVerilog(new SignMagConv(7)))

Elaborating design...
Done elaborating.
module SignMagConv(
  input        clock,
  input        reset,
  input        io_sign,
  input  [6:0] io_mag,
  output [7:0] io_twos
);
  wire [6:0] _T = ~io_mag; // @[cmd34.sc 8:20]
  wire [7:0] _T_1 = _T + 7'h1; // @[cmd34.sc 8:28]
  assign io_twos = io_sign ? _T_1 : {{1'd0}, io_mag}; // @[cmd34.sc 7:20 cmd34.sc 8:17 cmd34.sc 10:17]
endmodule



## Working With Bits

### Read-only access range `x(hi,lo)`
* Access nth bit of x `x(n)`
* _Note:_ can't assign extracted range

### Concatenation
* Combine signals x & y together `Cat(x,y)`

### Fill
* Repeat x, n times `Fill(n,x)`

In [37]:
class SignExtender(win: Int, wout: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(win.W))
        val out = Output(UInt(wout.W))
    })
    assert(win > 0)
    assert(win < wout)
    val signBit = io.in(win-1)
    val extension = Fill(wout-win, signBit)
    io.out := Cat(extension, io.in)
}
//printVerilog(new SignExtender(4,8))
println(getVerilog(new SignExtender(4,8)))

Elaborating design...
Done elaborating.
module SignExtender(
  input        clock,
  input        reset,
  input  [3:0] io_in,
  output [7:0] io_out
);
  wire  signBit = io_in[3]; // @[cmd36.sc 8:24]
  wire [3:0] extension = signBit ? 4'hf : 4'h0; // @[Bitwise.scala 72:12]
  assign io_out = {extension,io_in}; // @[Cat.scala 30:58]
endmodule



defined class SignExtender